Preprocess

In [1]:
# ===== BERT-OPTIMIZED TURKISH ADDRESS PREPROCESSING =====
import re
from unidecode import unidecode

def bert_optimized_address_preprocessing(text):
    """
    BERT modeli için optimize edilmiş Türkçe adres preprocessing fonksiyonu.
    Tüm preprocessing adımlarını birleştirir ve BERT'in en iyi performansı için tasarlanmıştır.
    """
    if not isinstance(text, str) or not text.strip():
        return ""
    
    # Orijinal metni koru (bazı analizler için)
    original_text = text
    
    # --- PHASE 1: INITIAL CLEANING ---
    # Küçük harfe çevir
    text = text.lower().strip()
    
    # Çoklu boşlukları ve tab/newline karakterlerini temizle
    text = re.sub(r'\s+', ' ', text)
    
    # --- PHASE 2: TURKISH CHARACTER NORMALIZATION ---
    # Türkçe karakterleri kontrollü şekilde normalize et (BERT türkçe modeli için optimize)
    turkish_normalization = {
        'ç': 'c', 'ğ': 'g', 'ı': 'i', 'ö': 'o', 'ş': 's', 'ü': 'u',
        'â': 'a', 'î': 'i', 'û': 'u', 'ô': 'o'
    }
    for tr_char, en_char in turkish_normalization.items():
        text = text.replace(tr_char, en_char)
    
    # --- PHASE 3: NUMBER & ALPHANUMERIC STANDARDIZATION ---
    # Sayı ve alfanümerik kodları standardize et (BERT tokenization için)
    
    # Alfanümerik kodları ayır: 24/A → 24 A, 12/B → 12 B
    text = re.sub(r'\b(\d+)\s*[/\-\\]\s*([a-zA-Z]\d*)\b', r'\1 \2', text)
    
    # Sayı kombinasyonlarını ayır: 24/5 → 24 5, 3/12 → 3 12  
    text = re.sub(r'\b(\d+)\s*[/\-\\]\s*(\d+)\b', r'\1 \2', text)
    
    # Noktalı sayıları ayır: 24.5 → 24 5 (BERT için)
    text = re.sub(r'\b(\d+)[.,](\d+)\b', r'\1 \2', text)
    
    # --- PHASE 4: COMPREHENSIVE ABBREVIATION EXPANSION ---
    # Türkiye'de yaygın kullanılan adres kısaltmaları
    abbreviations = {
        # Temel adres bileşenleri
        r'\bmah\.?\b': 'mahallesi',
        r'\bmahalle\b': 'mahallesi',
        r'\bmhl\.?\b': 'mahallesi',
        
        r'\bsok\.?\b': 'sokak',
        r'\bsk\.?\b': 'sokak',
        r'\bsokagi\b': 'sokak',
        
        r'\bcad\.?\b': 'caddesi',
        r'\bcd\.?\b': 'caddesi',
        r'\bcaddesi\b': 'caddesi',
        
        r'\bblv\.?\b': 'bulvari',
        r'\bbulvar\b': 'bulvari',
        r'\bblvr\.?\b': 'bulvari',
        
        r'\bno\.?\b': 'numara',
        r'\bnumara\b': 'numara',
        r'\bnum\.?\b': 'numara',
        
        # Yapı bileşenleri
        r'\bapt\.?\b': 'apartmani',
        r'\bapartman\b': 'apartmani',
        r'\bapart\.?\b': 'apartmani',
        
        r'\bd\.?\b': 'daire',
        r'\bdaire\b': 'daire',
        r'\bdr\.?\b': 'daire',
        
        r'\bk\.?\b': 'kat',
        r'\bkat\b': 'kat',
        r'\bkati\b': 'kat',
        
        r'\bblok\b': 'blok',
        r'\bbl\.?\b': 'blok',
        
        # Site ve kompleks
        r'\bsit\.?\b': 'sitesi',
        r'\bsite\b': 'sitesi',
        r'\bkonut\.?\b': 'konutlari',
        r'\bkompleks\b': 'kompleksi',
        r'\brezidans\b': 'rezidansi',
        
        # Diğer yaygın kısaltmalar
        r'\bpk\.?\b': 'park',
        r'\bpark\b': 'park',
        r'\bmez\.?\b': 'mezarligi',
        r'\bmeydan\b': 'meydani',
        r'\bmeyd\.?\b': 'meydani',
        r'\bçık\.?\b': 'cikmazı',
        r'\bçıkmaz\b': 'cikmazı'
    }
    
    for abbr_pattern, full_form in abbreviations.items():
        text = re.sub(abbr_pattern, full_form, text, flags=re.IGNORECASE)
    
    # --- PHASE 5: GEOGRAPHIC STANDARDIZATION ---
    # Yön ve konum belirteçleri
    directional_terms = {
        r'\bkuzey\b': 'kuzey',
        r'\bguney\b': 'guney',
        r'\bdogu\b': 'dogu',
        r'\bbati\b': 'bati',
        r'\biç\b': 'ic',
        r'\biçi\b': 'ic',
        r'\bdış\b': 'dis',
        r'\bdışı\b': 'dis',
        r'\balt\b': 'alt',
        r'\büst\b': 'ust',
        r'\bön\b': 'on',
        r'\barka\b': 'arka',
        r'\bsag\b': 'sag',
        r'\bsol\b': 'sol'
    }
    
    for direction_pattern, standardized in directional_terms.items():
        text = re.sub(direction_pattern, standardized, text, flags=re.IGNORECASE)
    
    # --- PHASE 6: CITY/DISTRICT NORMALIZATION ---
    # Büyük şehir ve ilçe isimleri standardizasyonu
    major_locations = {
        r'\bist\.?\b': 'istanbul',
        r'\bistanbul\b': 'istanbul',
        r'\bank\.?\b': 'ankara',
        r'\bankara\b': 'ankara',
        r'\bizm\.?\b': 'izmir',
        r'\bizmir\b': 'izmir',
        r'\bbeyoglu\b': 'beyoglu',
        r'\bkadikoy\b': 'kadikoy',
        r'\bbesiktas\b': 'besiktas',
        r'\buskudar\b': 'uskudar',
        r'\bsisli\b': 'sisli',
        r'\bfatih\b': 'fatih',
        r'\besenler\b': 'esenler',
        r'\bbahcelievler\b': 'bahcelievler'
    }
    
    for location_pattern, standardized in major_locations.items():
        text = re.sub(location_pattern, standardized, text, flags=re.IGNORECASE)
    
    # --- PHASE 7: NOISE REMOVAL ---
    # Gereksiz kelime ve ifadeleri kaldır
    noise_patterns = [
        r'\bmevki\b', r'\bmevkii\b', r'\byakini\b', r'\bcivarı\b',
        r'\bkarsisi\b', r'\bkarsisinda\b', r'\byani\b', r'\byaninda\b',
        r'\bonu\b', r'\bonunde\b', r'\barkasi\b', r'\barkasinda\b',
        r'\biçinde\b', r'\biçerisinde\b', r'\buzerinde\b', r'\baltinda\b'
    ]
    
    for noise_pattern in noise_patterns:
        text = re.sub(noise_pattern, '', text, flags=re.IGNORECASE)
    
    # --- PHASE 8: FEATURE EXTRACTION & TAGGING ---
    features = []
    
    # Posta kodu tespiti ve DOĞRUDAN EKLEME (generic tag yerine)
    postal_matches = re.findall(r'\b\d{5}\b', original_text)
    if postal_matches:
        # En büyük posta kodunu al (çoklu varsa en önemlisi genelde büyük olandır)
        main_postal = max(postal_matches)
        features.append(f'POSTAL_{main_postal}')
    
    # Apartman tespiti
    if re.search(r'\b(apt|apartman|apart)\b', original_text, re.IGNORECASE):
        features.append('APARTMENT')
    
    # Kat bilgisi tespiti
    if re.search(r'\b(\d+\.?\s*(kat|kati)|kat\s*\d+|(zemin|bodrum|cati)\s*kat)\b', original_text, re.IGNORECASE):
        features.append('FLOOR_INFO')
    
    # Daire/kapı numarası tespiti
    if re.search(r'\b(daire|d\.)\s*\d+\b', original_text, re.IGNORECASE):
        features.append('UNIT_NUMBER')
    
    # Site/kompleks tespiti
    if re.search(r'\b(site|sitesi|kompleks|kompleksi|rezidans|rezidansi|konut)\b', original_text, re.IGNORECASE):
        features.append('COMPLEX')
    
    # Blok bilgisi tespiti
    if re.search(r'\b(blok|bl\.?)\s*[a-zA-Z0-9]\b', original_text, re.IGNORECASE):
        features.append('BLOCK_INFO')
    
    # --- PHASE 9: FINAL CLEANUP ---
    # Özel karakterleri ve noktalamayı temizle (BERT için)
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Fazla boşlukları temizle
    text = re.sub(r'\s+', ' ', text).strip()
    
    # AKILLI KELİME FİLTRELEME: Anlamlı tek harfleri koru!
    words = text.split()
    filtered_words = []
    for i, word in enumerate(words):
        # Sayıları her zaman koru
        if word.isdigit():
            filtered_words.append(word)
        # Tek harf ama anlamlı ise koru (A blok, B kapısı vb.)
        elif len(word) == 1 and word.isalpha():
            # Önceki veya sonraki kelimeye bak
            context_words = []
            if i > 0:
                context_words.append(words[i-1].lower())
            if i < len(words) - 1:
                context_words.append(words[i+1].lower())
            
            # Blok, kapi, daire vb. bağlamında tek harf anlamlıdır
            meaningful_contexts = ['blok', 'kapi', 'kapisi', 'daire', 'apartmani', 'sitesi', 'mahallesi']
            if any(ctx in meaningful_contexts for ctx in context_words):
                filtered_words.append(word)
            # Yoksa çok kısa kelimeyi atla
        # Normal kelimeler (2+ karakter)
        elif len(word) >= 2:
            filtered_words.append(word)
    
    text = ' '.join(filtered_words)
    
    # --- PHASE 10: FEATURE INTEGRATION ---
    # Feature'ları metne entegre et (BERT için ek sinyal)
    if features:
        feature_string = ' '.join(features)
        text = f"{text} {feature_string}"
    
    # Final temizlik
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


In [2]:
# --- 1. Gerekli Kütüphanelerin Yüklenmesi ---
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
import gc
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score, confusion_matrix, classification_report

print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

# --- 2. Veri Yükleme ---
df_full = pd.read_csv('train.csv')

label_counts = df_full['label'].value_counts()
top_100_labels = label_counts.head(10390).index.tolist()
df = df_full[df_full['label'].isin(top_100_labels)].copy()
print(f"Veri seti: {len(df)} satır")

df = df.dropna(subset=['address', 'label'])
df['label'] = df['label'].astype(int)

# --- 3. Label Encoding ---
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['label'])
num_labels = len(label_encoder.classes_)
print(f"Sınıf sayısı: {num_labels}")

# --- 4. YENİ BERT-OPTIMIZED PREPROCESSING ---

# YENİ OPTIMIZE EDİLMİŞ PREPROCESSING KULLAN
print("📊 Yeni preprocessing uygulanıyor...")
df['address_bert_optimized'] = df['address'].apply(bert_optimized_address_preprocessing)

# Preprocessing karşılaştırması
print("\n📊 BERT-Optimized Preprocessing örnekleri:")
for i in range(5):
    original = df['address'].iloc[i]
    optimized = df['address_bert_optimized'].iloc[i]
    print(f"\n{i+1}. Örnek:")
    print(f"   📝 Orijinal:  '{original}'")
    print(f"   🎯 Optimized: '{optimized}'")
    print(f"   📏 Uzunluk:   {len(original)} → {len(optimized)} karakter")

# BERT-optimized veriyi kullan - TÜM VERİ İLE EĞİTİM
print("🎯 TÜM VERİ İLE EĞİTİM YAPILIYOR (VALIDATION YOK)")
X_full = df['address_bert_optimized'].tolist()
y_full = df['encoded_label'].tolist()

print(f"\n✅ BERT-optimized preprocessing tamamlandı!")
print(f"📊 Ortalama adres uzunluğu: {df['address_bert_optimized'].str.len().mean():.1f} karakter")
print(f"📊 FULL Training set: {len(X_full):,} samples")
print(f"📊 Validation set: NONE (tüm veri eğitimde kullanılıyor)")

# --- 5. BERT Model ---
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# --- 6. Tokenization ---
print("Tokenizasyon başlıyor - TÜM VERİ...")
full_encodings = tokenizer(X_full, truncation=True, padding=True, max_length=256)
print("Tokenizasyon tamamlandı.")

# --- 7. Dataset ---
class AddressDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

full_dataset = AddressDataset(full_encodings, y_full)
print(f"📊 Full dataset hazır: {len(full_dataset):,} samples")

CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Ti
Veri seti: 848237 satır
Sınıf sayısı: 10390
📊 Yeni preprocessing uygulanıyor...
Veri seti: 848237 satır
Sınıf sayısı: 10390
📊 Yeni preprocessing uygulanıyor...


KeyboardInterrupt: 

In [ ]:
# --- OPTIMIZED Class-wise Focal Loss Implementation ---
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer, TrainerCallback
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np
from collections import Counter

class ClassWiseFocalLoss(nn.Module):
    """
    OPTIMIZED Class-wise Focal Loss - Simplified for faster execution
    
    KEY OPTIMIZATIONS:
    - Simplified alpha calculation (no complex scaling)
    - Pre-computed alpha tensor
    - Reduced computational overhead
    - Faster initialization
    """
    
    def __init__(self, class_frequencies, gamma=1.5, alpha_mode='inverse_freq', reduction='mean'):
        super(ClassWiseFocalLoss, self).__init__()
        self.gamma = gamma
        self.reduction = reduction
        
        # SIMPLIFIED alpha hesaplama - daha hızlı
        if alpha_mode == 'inverse_freq':
            # Basit inverse frequency (no complex scaling)
            max_freq = max(class_frequencies)
            # Simple scaling: rare classes get higher alpha
            self.alphas = [min(max_freq / freq, 2.0) for freq in class_frequencies]
        else:
            # Default equal weighting
            self.alphas = [1.0] * len(class_frequencies)
            
        # Tensor'a çevir ve register et
        self.register_buffer('alpha_tensor', torch.FloatTensor(self.alphas))
        
        print(f"🎯 Focal Loss kuruldu: γ={gamma}, α_range={min(self.alphas):.2f}-{max(self.alphas):.2f}")
        
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        alpha_t = self.alpha_tensor.to(targets.device)[targets]
        focal_loss = alpha_t * (1-pt)**self.gamma * ce_loss
        
        return focal_loss.mean() if self.reduction == 'mean' else focal_loss

# OPTIMIZED Custom Trainer 
class ClassWiseFocalLossTrainer(Trainer):
    def __init__(self, *args, train_labels=None, gamma=1.5, alpha_mode='inverse_freq', **kwargs):
        super().__init__(*args, **kwargs)
        
        if train_labels is None:
            raise ValueError("train_labels gerekli")
            
        # SIMPLIFIED class frequency calculation
        label_counts = Counter(train_labels)
        max_label = max(label_counts.keys())
        class_frequencies = [label_counts.get(i, 1) for i in range(max_label + 1)]
        
        self.focal_loss = ClassWiseFocalLoss(
            class_frequencies=class_frequencies,
            gamma=gamma,
            alpha_mode=alpha_mode
        )
        
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss = self.focal_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

# SIMPLIFIED Training callback - Reduced computational overhead
class DetailedTrainingCallback(TrainerCallback):
    def __init__(self, label_encoder=None, num_classes=None):
        self.logged_steps = set()
        self.eval_count = 0
        # OPTIMIZATION: Reduce detailed analysis frequency
        self.detailed_analysis_frequency = 3  # Only every 3rd evaluation
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and 'learning_rate' in logs:
            step = state.global_step
            if step not in self.logged_steps and step % 2000 == 0:  # Log less frequently
                print(f"📈 Step {step}: LR = {logs['learning_rate']:.2e}")
                self.logged_steps.add(step)
    
    def on_evaluate(self, args, state, control, logs=None, model=None, **kwargs):
        """OPTIMIZED evaluation with reduced computational load"""
        if logs and model is not None:
            self.eval_count += 1
            print(f"\n🔍 EVAL #{self.eval_count} - STEP {state.global_step}")
            
            # Basic metrics always
            if 'eval_loss' in logs:
                print(f"📊 Loss: {logs['eval_loss']:.4f}")
            if 'eval_f1' in logs:
                print(f"📊 F1: {logs['eval_f1']:.4f}")
            
            # OPTIMIZATION: Detailed analysis only occasionally
            if self.eval_count % self.detailed_analysis_frequency == 0:
                print("🔍 Detailed analysis...")
                self._quick_metrics_analysis(model)
            else:
                print("⚡ Quick evaluation (detailed analysis skipped)")
    
    def _quick_metrics_analysis(self, model):
        """SIMPLIFIED metrics analysis - much faster"""
        try:
            # Just print basic info without heavy computation
            print(f"📊 Model on device: {next(model.parameters()).device}")
            print(f"📊 Model parameters: {sum(p.numel() for p in model.parameters()):,}")
            print("✅ Quick analysis complete")
        except Exception as e:
            print(f"⚠️ Analysis error: {str(e)[:50]}...")

# SIMPLIFIED Metrics function
def compute_enhanced_metrics(eval_pred):
    """OPTIMIZED metrics computation"""
    predictions, labels = eval_pred
    
    # Handle different input types efficiently
    if hasattr(predictions, 'logits'):
        predictions = predictions.logits
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    
    # Convert to numpy efficiently
    if hasattr(predictions, 'cpu'):
        predictions = predictions.cpu().numpy()
    if hasattr(labels, 'cpu'):
        labels = labels.cpu().numpy()
    
    # Get predictions and compute F1
    if len(predictions.shape) > 1:
        predictions = np.argmax(predictions, axis=-1)
    
    f1 = f1_score(labels, predictions, average='macro')
    return {'f1': f1}

# OPTIMIZED Logits preprocessing
def preprocess_logits_for_metrics(logits, labels):
    """Memory-efficient logits preprocessing"""
    return torch.argmax(logits, dim=-1), labels

print("✅ OPTIMIZED Focal Loss classes loaded! 🚀")
print("⚡ Initialization should be much faster now!")
print("🎯 Reduced computational overhead for training callback")

✅ OPTIMIZED Focal Loss classes loaded! 🚀
⚡ Initialization should be much faster now!
🎯 Reduced computational overhead for training callback


In [ ]:
# --- FULL DATA FINE-TUNING WITH FOCAL LOSS ---
print("🔄 GPU belleği temizleniyor ve model yükleniyor...")

# GPU belleğini temizle
if 'model' in locals():
    del model
if 'trainer' in locals():
    del trainer
torch.cuda.empty_cache()
gc.collect()
print("✅ GPU belleği temizlendi")

# Model yolunu belirle
model_path = r"C:\Users\bicer\OneDrive\Masaüstü\hepsiburada-hackathon-kaggle-etabi\results_focal_loss\checkpoint-106000"
print(f"📂 Model yükleniyor: {model_path}")

# Model ve tokenizer'ı yükle
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=num_labels)
model.to("cuda")

print("✅ Model yüklendi")

# EVALUATION İÇİN KÜÇÜK BİR VALİDATİON SET OLUŞTUR
# (Sadece metrics için - asıl eğitim full data ile)
from sklearn.model_selection import train_test_split

# Full data'dan %15'ini evaluation için ayır
X_train_focal, X_eval_focal, y_train_focal, y_eval_focal = train_test_split(
    X_full, y_full, test_size=0.15, random_state=42, stratify=y_full
)

print(f"📊 Focal Loss Training: {len(X_train_focal):,} samples")
print(f"📊 Evaluation (metrics için): {len(X_eval_focal):,} samples")

# Tokenization for Focal Loss
train_encodings_focal = tokenizer(X_train_focal, truncation=True, padding=True, max_length=256)
eval_encodings_focal = tokenizer(X_eval_focal, truncation=True, padding=True, max_length=256)

# Datasets
train_dataset_focal = AddressDataset(train_encodings_focal, y_train_focal)
eval_dataset_focal = AddressDataset(eval_encodings_focal, y_eval_focal)

print("✅ Focal Loss datasets hazır")

# FOCAL LOSS TRAINING AYARLARI
focal_training_args = TrainingArguments(
    output_dir='./final_full_training_focal',
    num_train_epochs=5,                    # Focal Loss için biraz daha fazla epoch
    per_device_train_batch_size=96,        # Focal Loss için optimize batch size
    per_device_eval_batch_size=96,         
    learning_rate=2e-4,                    # Focal Loss için optimize LR
    warmup_steps=8000,                     
    weight_decay=0.01,
    lr_scheduler_type="linear",      
    eval_strategy="steps",                 # Evaluation aktif
    eval_steps=1000,                       # Her 2000 step'te eval
    save_strategy="steps",           
    save_steps=1000,                       # Save daha seyrek
    load_best_model_at_end=True,           
    save_total_limit=3,                    
    metric_for_best_model="f1",            
    greater_is_better=True,                
    report_to=[],
    logging_steps=1000,
    fp16=True,
    seed=42,
    max_grad_norm=1.0
)

# Enhanced callback with detailed metrics
detailed_callback = DetailedTrainingCallback(
    label_encoder=label_encoder, 
    num_classes=num_labels
)

# Focal Loss trainer
focal_trainer = ClassWiseFocalLossTrainer(
    model=model,
    args=focal_training_args,
    train_dataset=train_dataset_focal,
    eval_dataset=eval_dataset_focal,
    compute_metrics=compute_enhanced_metrics,
    callbacks=[detailed_callback],
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    train_labels=y_train_focal,            # Sınıf bazlı alpha için gerekli
    gamma=1.5,                             # Optimal gamma
    alpha_mode='inverse_freq'              # Inverse frequency weighting
)

print("✅ Focal Loss trainer hazırlandı")
print(f"📊 Training samples: {len(train_dataset_focal):,}")
print(f"📊 Evaluation samples: {len(eval_dataset_focal):,}")
print(f"📊 Epoch sayısı: {focal_training_args.num_train_epochs}")
print(f"📊 Batch size: {focal_training_args.per_device_train_batch_size}")
print(f"📊 Evaluation: Her {focal_training_args.eval_steps} step'te")

# FOCAL LOSS TRAINING BAŞLAT
print("\n🚀 FOCAL LOSS TRAINING BAŞLIYOR...")
print("🎯 Her evaluation'da TP, FP, TN, FN detayları gösterilecek!")
print("="*60)

train_result = focal_trainer.train()

print("\n✅ FOCAL LOSS TRAINING TAMAMLANDI!")
print("="*60)
print("📈 Final Results:")
print(f"   • Training Loss: {train_result.training_loss:.4f}")
if hasattr(train_result, 'metrics'):
    for key, value in train_result.metrics.items():
        if isinstance(value, (int, float)):
            print(f"   • {key}: {value:.4f}")

# Final evaluation with detailed metrics
print("\n🔍 FINAL EVALUATION...")
eval_result = focal_trainer.evaluate()
print("📊 Final Evaluation Results:")
for key, value in eval_result.items():
    if isinstance(value, (int, float)):
        print(f"   • {key}: {value:.4f}")

# Final modeli kaydet
focal_save_path = "./kaggle_full_model_focal"
print(f"\n💾 Focal Loss model kaydediliyor: {focal_save_path}")
focal_trainer.save_model(focal_save_path)
tokenizer.save_pretrained(focal_save_path)

# Label encoder'ı da kaydet
import joblib
joblib.dump(label_encoder, f"{focal_save_path}/label_encoder.pkl")

print(f"✅ FOCAL LOSS TRAINING TAMAMLANDI!")
print(f"📁 Focal Loss Kaggle model: {focal_save_path}")
print(f"🎯 Bu model Class-wise Focal Loss ile eğitildi!")

🔄 GPU belleği temizleniyor ve model yükleniyor...
✅ GPU belleği temizlendi
📂 Model yükleniyor: C:\Users\bicer\OneDrive\Masaüstü\hepsiburada-hackathon-kaggle-etabi\results_focal_loss\checkpoint-106000
✅ Model yüklendi
✅ Model yüklendi
📊 Focal Loss Training: 721,001 samples
📊 Evaluation (metrics için): 127,236 samples
📊 Focal Loss Training: 721,001 samples
📊 Evaluation (metrics için): 127,236 samples


KeyboardInterrupt: 

# 📈 Training Visualization - Focal Loss Results

## Bu bölüm:
- Training/validation loss grafikleri
- F1 score gelişimi  
- Learning rate schedule
- Focal Loss eğitim özeti

In [ ]:
# --- FOCAL LOSS EĞİTİM GRAFİKLERİ ---
print("\n--- FOCAL LOSS EĞİTİM GRAFİKLERİ ÇİZİLİYOR ---")

import matplotlib.pyplot as plt
import pandas as pd

try:
    # Training history'den verileri çek
    history = focal_trainer.state.log_history
    
    # Training ve validation verilerini ayır
    train_logs = [log for log in history if 'loss' in log and 'eval_loss' not in log]
    eval_logs = [log for log in history if 'eval_loss' in log]
    
    # Veri çerçeveleri oluştur
    train_df = pd.DataFrame(train_logs)
    eval_df = pd.DataFrame(eval_logs)
    
    # 2x2 subplot oluştur
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle('🎯 FOCAL LOSS Fine-tuning Eğitim Sonuçları', fontsize=16, fontweight='bold')
    
    # 1. Training vs Validation Loss (Aynı grafikte)
    if not train_df.empty and 'loss' in train_df.columns:
        ax1.plot(train_df['step'], train_df['loss'], 'b-', linewidth=2, label='Training Loss', alpha=0.8)
    if not eval_df.empty and 'eval_loss' in eval_df.columns:
        ax1.plot(eval_df['step'], eval_df['eval_loss'], 'r-', linewidth=2, label='Validation Loss', alpha=0.8)
    
    ax1.set_title('🔥 Focal Loss: Training vs Validation', fontweight='bold')
    ax1.set_xlabel('Step')
    ax1.set_ylabel('Loss')
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # En düşük loss'ları göster
    if not train_df.empty and 'loss' in train_df.columns:
        min_train_loss = train_df['loss'].min()
        ax1.axhline(y=min_train_loss, color='blue', linestyle='--', alpha=0.5)
    if not eval_df.empty and 'eval_loss' in eval_df.columns:
        min_val_loss = eval_df['eval_loss'].min()
        ax1.axhline(y=min_val_loss, color='red', linestyle='--', alpha=0.5)
    
    # 2. F1 Score Progression
    if not eval_df.empty and 'eval_f1' in eval_df.columns:
        ax2.plot(eval_df['step'], eval_df['eval_f1'], 'g-', linewidth=3, label='F1 Score', marker='o', markersize=4)
        ax2.set_title('🎯 F1 Score Gelişimi (Validation)', fontweight='bold')
        ax2.set_xlabel('Step')
        ax2.set_ylabel('F1 Score')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        # En iyi F1 score'u göster
        best_f1 = eval_df['eval_f1'].max()
        best_step = eval_df.loc[eval_df['eval_f1'].idxmax(), 'step']
        ax2.axhline(y=best_f1, color='red', linestyle='--', alpha=0.7)
        ax2.text(0.02, 0.95, f'🏆 En İyi F1: {best_f1:.4f}\n📍 Step: {best_step}', 
                transform=ax2.transAxes, fontsize=10, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
        
        # F1 score range'ini optimize et
        f1_min = eval_df['eval_f1'].min()
        f1_max = eval_df['eval_f1'].max()
        f1_margin = (f1_max - f1_min) * 0.1
        ax2.set_ylim(max(0, f1_min - f1_margin), min(1, f1_max + f1_margin))
    
    # 3. Learning Rate Schedule
    if not train_df.empty and 'learning_rate' in train_df.columns:
        ax3.plot(train_df['step'], train_df['learning_rate'], 'purple', linewidth=2, label='Learning Rate')
        ax3.set_title('📉 Learning Rate Schedule', fontweight='bold')
        ax3.set_xlabel('Step')
        ax3.set_ylabel('Learning Rate')
        ax3.grid(True, alpha=0.3)
        ax3.legend()
        ax3.ticklabel_format(style='scientific', axis='y', scilimits=(0,0))
        
        # LR değişimi göster
        initial_lr = train_df['learning_rate'].iloc[0]
        final_lr = train_df['learning_rate'].iloc[-1]
        ax3.text(0.02, 0.95, f'🚀 Başlangıç: {initial_lr:.2e}\n🎯 Son: {final_lr:.2e}\n📉 Azalma: {(final_lr/initial_lr)*100:.1f}%', 
                transform=ax3.transAxes, fontsize=9, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lavender', alpha=0.8))
    
    plt.tight_layout()
    
    # Dosya adını focal loss için güncelle
    plt.savefig('focal_loss_training_results.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # FOCAL LOSS ÖZET İSTATİSTİKLER
    print("\n" + "="*70)
    print("🎯 FOCAL LOSS EĞİTİM ÖZETİ")
    print("="*70)
    
    if not train_df.empty:
        final_train_loss = train_df['loss'].iloc[-1]
        min_train_loss = train_df['loss'].min()
        print(f"🔹 Son Training Loss: {final_train_loss:.4f}")
        print(f"🔹 En Düşük Training Loss: {min_train_loss:.4f}")
    
    if not eval_df.empty:
        final_val_loss = eval_df['eval_loss'].iloc[-1]
        min_val_loss = eval_df['eval_loss'].min()
        best_f1 = eval_df['eval_f1'].max()
        final_f1 = eval_df['eval_f1'].iloc[-1]
        best_f1_step = eval_df.loc[eval_df['eval_f1'].idxmax(), 'step']
        
        print(f"🔹 Son Validation Loss: {final_val_loss:.4f}")
        print(f"🔹 En Düşük Validation Loss: {min_val_loss:.4f}")
        print(f"🔹 🏆 En İyi F1 Score: {best_f1:.4f} (Step {best_f1_step})")
        print(f"🔹 Son F1 Score: {final_f1:.4f}")
        
        # F1 improvement
        if len(eval_df) > 1:
            first_f1 = eval_df['eval_f1'].iloc[0]
            f1_improvement = best_f1 - first_f1
            print(f"🔹 📈 F1 İyileşmesi: +{f1_improvement:.4f} ({f1_improvement/first_f1*100:.1f}%)")
    
    if not train_df.empty and 'learning_rate' in train_df.columns:
        final_lr = train_df['learning_rate'].iloc[-1]
        initial_lr = train_df['learning_rate'].iloc[0]
        print(f"🔹 Başlangıç Learning Rate: {initial_lr:.2e}")
        print(f"🔹 Son Learning Rate: {final_lr:.2e}")
        print(f"🔹 LR Azalma Oranı: {(final_lr/initial_lr)*100:.1f}%")
    
    # Focal Loss özellikleri
    print(f"\n🎯 FOCAL LOSS ÖZELLİKLERİ:")
    print(f"🔹 Gamma (γ): {focal_trainer.focal_loss.gamma}")
    print(f"🔹 Alpha Mode: {focal_trainer.focal_loss.alpha_mode}")
    print(f"🔹 Alpha Range: {min(focal_trainer.focal_loss.alphas):.3f} - {max(focal_trainer.focal_loss.alphas):.3f}")
    print(f"🔹 Training Samples: {len(train_dataset_focal):,}")
    print(f"🔹 Validation Samples: {len(eval_dataset_focal):,}")
    print(f"🔹 Batch Size: {focal_training_args.per_device_train_batch_size}")
    print(f"🔹 Epochs: {focal_training_args.num_train_epochs}")
    
    print("="*70)
    print("✅ Focal Loss grafikleri 'focal_loss_training_results.png' olarak kaydedildi!")
    print("🎯 Class-wise Focal Loss ile eğitim başarıyla tamamlandı!")
    
except Exception as e:
    print(f"❌ Grafik oluşturma hatası: {e}")
    print("⚠️ Training history verilerine erişilemedi.")
    
    # Alternatif basit özet
    try:
        if 'train_result' in locals():
            print(f"\n📊 Basit Eğitim Özeti:")
            print(f"🔹 Final Training Loss: {train_result.training_loss:.4f}")
            if hasattr(train_result, 'metrics'):
                for key, value in train_result.metrics.items():
                    if isinstance(value, (int, float)):
                        print(f"🔹 {key}: {value:.4f}")
    except:
        print("⚠️ Özet bilgiler de alınamadı.")